---
title: "Model-Agnostic Meta-Learning for Fast Adaptation of Deep Networks"
date: 2025-11-14       
description: "메타 러닝 관련 논문 요약 및 주요 내용"
categories: [MetaLearning, MAML, Review]
author: "김한울"
---

`업데이트 내역`

||||
|-|-|-|
|2025-11-14|첫 게시|서론 심화 업데이트|
|2025-11-16|섹션 2.1 업데이트||

```
@inproceedings{finn2017model,
  title={Model-agnostic meta-learning for fast adaptation of deep networks},
  author={Finn, Chelsea and Abbeel, Pieter and Levine, Sergey},
  booktitle={International conference on machine learning},
  pages={1126--1135},
  year={2017},
  organization={PMLR}
}
```

# 초록 (Abstract)

본 논문은 경사 하강법(gradient descent)으로 훈련되는 어떠한 모델과도 호환되며 분류, 회귀, 강화학습 등 다양한 학습 문제에 적용할 수 있다는 점에서 **모델에 구애받지 않는(model-agnostic) 메타 러닝 알고리즘**을 제안합니다. 메타 러닝의 목표는 다양한 학습 과제(task)에 대해 모델을 훈련함으로써, 적은 수의 훈련 샘플만으로도 새로운 과제를 해결할 수 있도록 하는 것입니다.

우리가 제안하는 방식은 새로운 과제에 대한 소량의 훈련 데이터와 몇 번의 경사 하강 단계(gradient step)만으로도 해당 과제에서 우수한 일반화 성능을 달성할 수 있도록 모델의 파라미터를 명시적으로 훈련합니다. 다시 말해, 본 방법은 모델이 **미세조정(fine-tuning)에 용이하도록 훈련**하는 것입니다.

본 논문은 이 접근법이 두 개의 퓨샷(few-shot) 이미지 분류 벤치마크에서 최고 수준(state-of-the-art)의 성능을 달성하고, 퓨샷 회귀 문제에서도 우수한 결과를 보이며, 신경망 정책을 사용하는 정책 경사(policy gradient) 강화학습의 미세조정 속도를 크게 향상시킨다는 것을 입증합니다.

# 1. 서론 (Introduction)

## 인간 지능의 핵심적인 특징과 AI agent에게 바라는 것

+ 적은 수의 예시만으로 사물을 인식하거나, 단 몇 분의 경험만으로 새로운 기술을 습득하는 등, 빠르게 학습하는 능력
+ 우리가 만드는 인공 에이전트 역시 인간의 지능과 같은 특징을 가지면 좋다.   

즉, 소수의 예시만으로도 신속하게 학습하고 적응하며, 더 많은 데이터가 주어짐에 따라 지속적으로 적응해 나가야 함. 

## 빠르고 유연한 학습은 어려운 과제

+ 에이전트는 새로운 데이터에 과적합(overfitting)되는 것을 피해야 하고, 
+ 기존의 경험을 소량의 새로운 정보와 통합해야 한다. 
+ 더욱이, 사전 경험(prior experiments)과 새로운 데이터의 형태는 과제(task)에 따라 달라진다.    

따라서 최대한의 적용 가능성을 확보하기 위해서는, '학습하는 방법을 학습'하는 메커니즘(즉, 메타 러닝)이 특정 과제나 연산 형태에 국한되지 않고 범용적(general)이어야 합니다.

## 제안: **모델에 구애받지 않는(model-agnostic)** 메타 러닝 알고리즘

+ 경사 하강법(gradient descent)으로 훈련되는 어떠한 학습 문제와 모델에도 직접 적용될 수 있다는 의미. 
+ 심층 신경망 모델(deep neural network models)에 초점을 맞추고 있긴 하지만, 본 논문이 제안하는 접근법(MAML)은 최소한의 수정만으로도 분류, 회귀, 정책 경사(policy gradient) 강화학습 등 다양한 아키텍처와 문제 설정에 얼마나 쉽게 적용될 수 있는지를 보여줌. 

메타 러닝에서 훈련된 모델의 목표는 적은 양의 새로운 데이터만으로 새로운 과제를 신속하게 학습하는 것이며, 모델은 메타 학습기에 의해 수많은 다른 과제에 대해 학습할 수 있도록 훈련됩니다.

## 핵심 아이디어 of MAML   

새로운 과제에서 얻은 소량의 데이터로 계산된 한 번 이상의 경사 하강 단계를 거쳐 파라미터가 업데이트되었을 때, 해당 과제에서 모델의 성능이 극대화되도록 모델의 **초기 파라미터**를 훈련하는 것입니다. 

+ 업데이트 함수나 학습 규칙을 학습하는 기존의 메타 러닝 방법들[1]과 달리, MAML 알고리즘은 학습해야 할 파라미터의 수를 늘리지 않으며, 순환 모델(recurrent model) [2]이나 샴 네트워크(Siamese network)[3]를 요구하는 것처럼 모델 아키텍처에 제약을 가하지도 않습니다. 
    + [1] (Schmidhuber, 1987; Bengio et al., 1992; Andrychowicz et al., 2016; Ravi & Larochelle, 2017)   
    + [2] (Santoro et al., 2016)    
    + [3] (Koch, 2015)   
+ 또한, 완전 연결(fully connected), 컨볼루션(convolutional), 순환(recurrent) 신경망 등과 쉽게 결합할 수 있습니다. 
+ 미분 가능한 지도 학습 손실 함수는 물론, 미분 불가능한 강화학습 목표 함수를 포함한 다양한 종류의 손실 함수와도 함께 사용할 수 있습니다.


단 몇 번의 경사 하강 단계만으로, 혹은 단 한 번의 단계만으로도 새로운 과제에서 좋은 결과를 낼 수 있도록 모델의 파라미터를 훈련하는 과정은, **특징 학습(feature learning)**의 관점에서 볼 때 여러 과제에 폭넓게 적용 가능한 내부 표현(internal representation)을 구축하는 것으로 해석할 수 있습니다.    

+ 만약 내부 표현이 여러 과제에 적합하다면, 파라미터를 약간만 미세조정하는 것(예: 피드포워드 모델의 마지막 레이어 가중치를 주로 수정하는 것)만으로도 좋은 결과를 얻을 수 있습니다.   
+ 결과적으로, 우리 절차는 쉽고 빠르게 미세조정될 수 있는 모델을 최적화하여, 신속한 학습에 적합한 공간에서 적응이 일어나도록 만듭니다. 

**동적 시스템(dynamical systems)** 관점에서 보면, 우리의 학습 과정은 파라미터에 대한 새로운 과제들의 손실 함수의 민감도(sensitivity)를 극대화하는 것으로 볼 수 있습니다.    

+ 민감도가 높을 때, 파라미터의 작은 국소적 변화가 과제 손실(task loss)을 크게 개선할 수 있기 때문입니다.   

> maximizing the sensitivity of the loss functions of new tasks with respect to the parameters?


## 이 연구의 주된 기여   
+ 적은 횟수의 경사 하강 업데이트만으로 새로운 과제에 대한 빠른 학습이 가능하도록 모델의 파라미터를 훈련하는, 단순하면서도 모델과 과제에 구애받지 않는 메타 러닝 알고리즘. 
+ 이 알고리즘을 완전 연결 신경망과 컨볼루션 신경망을 포함한 다양한 모델 유형과, 퓨샷(few-shot) 회귀, 이미지 분류, 강화학습 등 여러 영역에서 시연. 

평가는 제안된 메타 러닝 알고리즘이 지도 분류를 위해 특별히 설계된 최신 원샷(one-shot) 학습 방법들과 비교하여 더 적은 파라미터를 사용하면서도 우수한 성능을 보이며, 회귀 문제에도 쉽게 적용될 수 있고, 과제 가변성이 존재하는 상황에서 강화학습을 가속화하여 초기화 방식으로서의 직접적인 사전 훈련(pretraining)보다 월등히 뛰어난 성능을 보인다는 것을 보여줍니다.

# 1.0 서론 짚고 넘어가기

> **maximizing the sensitivity of the loss functions**

## 핵심 개념: 민감도(Sensitivity) = 기울기(Gradient)의 크기

수학, 특히 최적화 문제에서 어떤 함수의 "파라미터에 대한 민감도"는 **기울기(gradient)**로 표현됩니다. 기울기는 파라미터를 아주 약간 변경했을 때 함수 값이 얼마나, 그리고 어느 방향으로 변하는지를 나타냅니다.

- **기울기의 방향**: 함수 값이 가장 가파르게 증가하는 방향
- **기울기의 크기(magnitude)**: 그 가파른 정도. 즉, **민감도**

따라서 "파라미터(θ)에 대한 새로운 과제의 손실 함수(L)의 민감도를 극대화한다"는 것은, 손실 함수의 기울기 벡터 $\nabla_{\theta} \mathcal{L}$의 크기(norm), 즉 $\Vert \nabla_{\theta} \mathcal{L} \Vert$ 를 크게 만드는 것을 의미합니다.

## 수식을 통한 설명

1.  **기본 설정**
* 모델 파라미터: $\theta$
* 새로운 (임의의) 과제 $\mathcal{T}_i$에 대한 손실 함수: $\mathcal{L}_{\mathcal{T}_i}(\theta)$

2.  **민감도(Sensitivity)의 수학적 표현**   

파라미터 $\theta$에 대한 손실 함수 $\mathcal{L}_{\mathcal{T}_i}$의 민감도는 기울기(gradient)의 크기(norm)로 나타낼 수 있습니다.

$$ \text{Sensitivity} = \| \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta) \| $$

+ 이 값이 크다는 것은 손실 함수의 "경사면"이 매우 가파르다는 것을 의미합니다.

3.  **파라미터의 작은 변화와 손실의 큰 개선**

경사 하강법에서는 파라미터를 다음과 같이 업데이트합니다.

$$ \theta' = \theta - \alpha \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta) $$

여기서 $\alpha$는 학습률(learning rate)입니다.

+ 이때, 업데이트 후의 손실 값 $\mathcal{L}_{\mathcal{T}_i}(\theta')$는 1차 테일러 근사(first-order Taylor approximation)를 통해 다음과 같이 예측할 수 있습니다.

$$ \mathcal{L}_{\mathcal{T}_i}(\theta') \approx \mathcal{L}_{\mathcal{T}_i}(\theta) + (\theta' - \theta)^T \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta) $$

위 식에 $\theta' - \theta = -\alpha \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta)$를 대입하면,

$$ \mathcal{L}_{\mathcal{T}_i}(\theta') \approx \mathcal{L}_{\mathcal{T}_i}(\theta) + (-\alpha \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta))^T \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta) $$

$$ \mathcal{L}_{\mathcal{T}_i}(\theta') \approx \mathcal{L}_{\mathcal{T}_i}(\theta) - \alpha (\nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta))^T (\nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta)) $$

벡터와 자신의 내적(dot product)은 크기의 제곱이므로,

$$ \mathcal{L}_{\mathcal{T}_i}(\theta') \approx \mathcal{L}_{\mathcal{T}_i}(\theta) - \alpha \| \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta) \|^2 $$

4.  **결론: 손실 개선량**

위 식을 정리하여 한 번의 업데이트로 인한 **손실 개선량**을 살펴보면 다음과 같습니다.

$$ \text{Loss Improvement} = \mathcal{L}_{\mathcal{T}_i}(\theta) - \mathcal{L}_{\mathcal{T}_i}(\theta') \approx \alpha \| \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta) \|^2 $$

이 수식은 매우 중요한 점을 시사합니다.

> **손실 개선량은 민감도(기울기의 크기)의 제곱에 비례합니다.**

따라서 MAML의 학습 과정은 어떤 새로운 과제 $\mathcal{T}_i$가 주어지더라도, 현재 파라미터 $\theta$ 위치에서 손실 함수의 기울기 크기 $\vert \nabla_\theta \mathcal{L}_{\mathcal{T}_i}(\theta)\vert$가 크도록 만드는 것입니다. 

+ 이렇게 되면 단 한 번의 경사 하강 단계만으로도 손실 값을 **크게** 줄일 수 있어, 빠르고 효율적인 적응(adaptation)이 가능해집니다.

## 비유를 통한 이해

-   **나쁜 초기 파라미터 (낮은 민감도)**: 넓은 고원이나 평지에 서 있는 것과 같습니다. 어느 방향으로 한 걸음 내딛어도 고도(손실)는 거의 변하지 않습니다. 최저점(최적해)에 도달하려면 수많은 걸음을 옮겨야 합니다.
-   **좋은 초기 파라미터 (높은 민감도, MAML의 목표)**: 여러 계곡(각 과제의 최적해)으로 내려가는 길이 시작되는 **산등성이의 중앙**에 서 있는 것과 같습니다. 어떤 계곡으로 내려가야 할지 목표가 주어지면, 그 방향은 매우 가파르기 때문에 단 몇 걸음만으로도 고도(손실)를 크게 낮출 수 있습니다.

MAML은 바로 이 "산등성이의 중앙"과 같은 초기 파라미터 $\theta$를 찾는 알고리즘이라고 할 수 있습니다.

# 2. 모델에 구애받지 않는 메타 러닝 (Model-Agnostic Meta-Learning)

우리는 신속한 적응(rapid adaptation)이 가능한 모델을 훈련하고자 하며, 이러한 문제 설정은 종종 퓨샷 학습(few-shot learning)으로 공식화됩니다. 본 섹션에서는 문제 설정을 정의하고, 우리가 제안하는 알고리즘의 일반적인 형태를 제시하겠습니다.

## 2.1. 메타 러닝 문제 설정 (Meta-Learning Problem Set-Up)

**The Goal of few-shot meta-learning?**

퓨샷 메타 러닝의 목표는 단 몇 개의 데이터포인트와 훈련 반복만으로 새로운 과제에 신속하게 적응할 수 있는 모델을 훈련하는 것입니다. 

+ 이를 위해, 모델 또는 학습기는 메타-학습 단계에서 여러 과제(task)에 대해 훈련을 받습니다. 
+ 그 결과 훈련된 모델은 적은 수의 예시나 시도만으로도 새로운 과제에 빠르게 적응할 수 있게 됩니다. 

결과적으로, 메타 러닝 문제는 개별 과제 전체를 하나의 훈련 예시로 취급하는 셈입니다. 본 섹션에서는 다양한 학습 영역의 간략한 예시를 포함하여 이 메타 러닝 문제 설정을 일반적인 방식으로 공식화합니다. 섹션 3에서는 두 가지 다른 학습 영역을 자세히 논의할 것입니다.

우리는 관측값 $\mathbf{x}$를 출력값 **a**로 매핑하는 모델 $f$를 고려합니다. 
메타 러닝 과정에서, 모델은 수많은 혹은 무한한 수의 과제에 적응할 수 있도록 훈련됩니다. 

+ 우리는 분류에서 강화학습에 이르기까지 다양한 학습 문제에 우리의 프레임워크를 적용하고자 하므로, 아래에 학습 과제에 대한 일반적인 개념을 도입합니다. 공식적으로, 각 과제 $\mathcal{T}$는 다음과 같이 구성됩니다.

$$\mathcal{T} = \{\mathcal{L}(x_1, a_1, ..., x_H, a_H), q(x_1), q(x_{t+1}|x_t, a_t), H\}$$

+ $\mathcal{L}$: 손실함수
+ $q(x_1)$: 초기 관측값의 분포
+ $q(x_{t+1}|x_t, a_t)$: 전이 분포
+ $H$: 에피소드 길이

i.i.d.에서, 지도 학습 문제에서는 에피소드 길이가 $H=1$입니다. 
모델은 각 시점 $t$에서 출력값 $a_t$를 선택하여 길이 $H$의 샘플을 생성할 수 있습니다. 

손실 함수 $\mathcal{L}(x_1, a_1, ..., x_H, a_H) \rightarrow \mathbb{R}$은 과제별 피드백을 제공하며, 이는 마르코프 결정 과정(Markov decision process)에서의 오분류 손실(misclassification loss)이나 비용 함수(cost function)의 형태일 수 있습니다.


우리의 메타 러닝 시나리오에서는, 모델이 적응해야 할 과제들의 분포 $p(\mathcal{T})$를 고려합니다. 
K-샷(K-shot) 학습 설정에서, 모델은 $p(\mathcal{T})$로부터 추출된 새로운 과제 $\mathcal{T}_i$를 학습하도록 훈련됩니다. 이때 학습에는 $q_i$에서 추출된 단 K개의 샘플과, $\mathcal{T}_i$에 의해 생성된 피드백(손실) $\mathcal{L}_{\mathcal{T}_i}$만이 사용됩니다. 메타-훈련(meta-training) 중에는 $p(\mathcal{T})$에서 과제 $\mathcal{T}_i$가 샘플링되고, 모델은 K개의 샘플과 해당 손실 $\mathcal{L}_{\mathcal{T}_i}$로부터의 피드백으로 훈련된 후, $\mathcal{T}_i$의 새로운 샘플에 대해 테스트됩니다. 모델 $f$는 $q_i$의 새로운 데이터에 대한 테스트 오차(test error)가 파라미터에 따라 어떻게 변하는지를 고려하여 개선됩니다. 결과적으로, 샘플링된 과제 $\mathcal{T}_i$에 대한 테스트 오차가 메타-학습 과정의 훈련 오차(training error) 역할을 하게 됩니다. 메타-훈련이 끝나면, $p(\mathcal{T})$에서 새로운 과제들이 샘플링되고, K개의 샘플로부터 학습한 후의 모델 성능으로 메타-성능(meta-performance)이 측정됩니다. 일반적으로, 메타-테스트에 사용되는 과제들은 메타-훈련 과정에서는 사용되지 않습니다(held out).

## 2.1.0 상세 설명

해당 섹션은 MAML이 해결하고자 하는 문제, 
즉 **퓨샷 메타 러닝(Few-shot Meta-Learning)**을 수학적으로 엄밀하게 정의합니다. 핵심은 기존의 머신러닝처럼 '데이터포인트'를 학습하는 것이 아니라, '과제(Task)' 자체를 하나의 학습 단위로 보고, 새로운 과제에 빠르게 적응하는 능력을 학습하는 것입니다.

### 1. 과제($\mathcal{T}$)의 일반적 정의

MAML은 분류, 회귀, 강화학습 등 다양한 문제에 적용 가능하도록 '과제'를 매우 일반적인 형태로 정의합니다.

> $\mathcal{T} = \{\mathcal{L}(x_1, a_1, ..., x_H, a_H), q(x_1), q(x_{t+1}|x_t, a_t), H\}$

이 수식의 각 구성 요소는 다음과 같은 의미를 가집니다.

*   **$\mathcal{L}(\dots)$ : 손실 함수 (Loss Function)**
    *   **의미**: 해당 과제($\mathcal{T}$)에서 모델의 성능을 평가하는 기준입니다. 즉, "무엇을 잘해야 하는가?"를 정의합니다.
    *   **예시**:
        *   **분류**: Cross-Entropy Loss (예측 레이블과 실제 레이블 간의 차이)
        *   **회귀**: Mean Squared Error (MSE) (예측값과 실제값 간의 제곱 오차)
        *   **강화학습**: 에피소드 동안 받은 보상(Reward)의 합에 음수를 취한 값 (보상을 최대화하는 것이 목표이므로, 손실은 음수 보상으로 정의)

*   **$q(x_1)$ : 초기 상태 분포 (Initial State Distribution)**
    *   **의미**: 과제가 시작될 때, 첫 번째 관측값(데이터) $x_1$이 어떤 분포를 따르는지를 나타냅니다.
    *   **예시**:
        *   **지도학습 (분류/회귀)**: 전체 데이터셋에서 입력 데이터(예: 이미지, 문장)를 샘플링하는 분포입니다.
        *   **강화학습**: 에이전트가 에피소드를 시작하는 초기 상태(예: 미로의 시작점, 로봇의 초기 자세)의 분포입니다.

*   **$q(x_{t+1}|x_t, a_t)$ : 상태 전이 분포 (State Transition Distribution)**
    *   **의미**: 현재 상태 $x_t$에서 모델이 행동 $a_t$를 취했을 때, 다음 상태 $x_{t+1}$이 어떻게 될지를 결정하는 환경의 동역학(dynamics)입니다.
    *   **예시**:
        *   **지도학습 (분류/회귀)**: 이 부분은 **적용되지 않습니다**. 지도학습은 데이터가 독립적이고 순서가 없으며, 모델의 출력이 다음 입력에 영향을 주지 않기 때문입니다. 논문에서 언급하듯, 이 경우 에피소드 길이 $H=1$로 간주하여 상태 전이를 무시합니다.
        *   **강화학습**: 환경의 물리 법칙이나 게임의 규칙에 해당합니다. (예: 로봇이 '오른쪽으로 힘주기'라는 행동 $a_t$를 취하면, 물리 엔진에 의해 다음 자세 $x_{t+1}$이 결정됩니다.)

*   **$H$ : 에피소드 길이 (Horizon)**
    *   **의미**: 하나의 과제를 수행하는 데 걸리는 시간 스텝의 길이입니다.
    *   **예시**:
        *   **지도학습 (분류/회귀)**: $H=1$ 입니다. 입력 하나에 대해 출력 하나를 내놓고 과제가 종료됩니다.
        *   **강화학습**: 미로 찾기에서 목표에 도달하거나, 정해진 스텝(예: 100 스텝)이 지날 때까지의 길이입니다.

### 2. 메타 러닝의 전체 과정

MAML은 단일 과제 $\mathcal{T}$를 푸는 것이 아니라, 수많은 과제들의 **분포 $p(\mathcal{T})$**로부터 학습합니다.

*   **$p(\mathcal{T})$ : 과제들의 분포 (Distribution over Tasks)**
    *   **의미**: 풀고자 하는 문제들의 '종류' 또는 '집합'을 의미합니다. 개별 데이터의 분포가 아니라, **학습 문제 자체의 분포**입니다.
    *   **예시**:
        *   **이미지 분류**: "Omniglot 데이터셋에서 5개의 클래스를 무작위로 뽑아, 이 5개를 분류하는 모든 가능한 문제들의 집합"
        *   **회귀**: "진폭과 위상이 특정 범위 내에서 무작위로 변하는 모든 사인 함수(sinusoid)를 맞추는 문제들의 집합"
        *   **강화학습**: "목표 지점이 단위 정사각형 내에서 무작위로 설정되는 모든 2D 길찾기 문제들의 집합"

#### 메타-훈련 (Meta-Training) 단계

모델이 "빠르게 적응하는 법"을 배우는 과정입니다.

1.  **과제 샘플링**: 과제 분포 $p(\mathcal{T})$에서 하나의 과제 $\mathcal{T}_i$를 뽑습니다. (예: '개', '고양이', '새'를 분류하는 문제를 하나 만듭니다.)

2.  **K-샷 데이터 샘플링 (Support Set)**: 샘플링된 과제 $\mathcal{T}_i$에서 **적응(adaptation)을 위한** K개의 데이터 샘플을 뽑습니다. (예: '개', '고양이', '새' 사진을 각각 K장씩 뽑습니다.)

3.  **내부 학습 (Inner-loop Update)**: 현재 모델 파라미터 $\theta$를 K개의 샘플을 사용하여 **임시로** 업데이트합니다. 이 과정은 새로운 과제에 대한 빠른 적응을 모방하는 단계입니다. (MAML에서는 1회 이상의 경사 하강을 통해 임시 파라미터 $\theta'_i$를 계산합니다.)

4.  **테스트 데이터 샘플링 (Query Set)**: **동일한 과제** $\mathcal{T}_i$에서 **평가를 위한** 새로운 데이터 샘플들을 뽑습니다. (예: 앞서 사용하지 않은 '개', '고양이', '새' 사진들을 추가로 뽑습니다.)

5.  **메타-최적화 (Meta-Optimization)**: **임시로 업데이트된** 파라미터 $\theta'_i$를 사용하여 테스트 데이터에 대한 손실 $\mathcal{L}_{\mathcal{T}_i}(\theta'_i)$를 계산합니다. 이 손실을 줄이는 방향으로 **원본 파라미터 $\theta$**를 업데이트합니다.
    *   **핵심 아이디어**: 이 과정은 "어떻게 $\theta$를 수정해야, 한 번의 경사 하강만으로도 새로운 과제를 더 잘 풀 수 있을까?"를 학습하는 것과 같습니다. 즉, **적응 후의 성능**을 기준으로 **적응 전의 파라미터**를 최적화합니다.

이 1~5번 과정을 수많은 다른 과제 $\mathcal{T}_j, \mathcal{T}_k, \dots$에 대해 반복합니다.

#### 메타-테스트 (Meta-Testing) 단계

학습된 모델의 일반화 성능, 즉 "처음 보는 과제에 얼마나 빠르게 적응하는가?"를 평가하는 과정입니다.

1.  **새로운 과제 샘플링**: 메타-훈련 과정에서 **한 번도 보지 못했던** 새로운 과제 $\mathcal{T}_{test}$를 $p(\mathcal{T})$에서 뽑습니다.

2.  **K-샷 적응**: $\mathcal{T}_{test}$에서 K개의 샘플을 뽑아 모델을 딱 한 번 (또는 몇 번) 업데이트합니다.

3.  **최종 평가**: $\mathcal{T}_{test}$의 새로운 데이터에 대해 적응된 모델의 최종 성능을 측정합니다.

이처럼 MAML의 문제 설정은, 특정 과제에 대한 전문가 모델을 만드는 것이 아니라, 어떤 과제가 주어지더라도 K개의 예시만 보고 빠르게 전문가 수준으로 적응할 수 있는 **"준비된 초기 모델 파라미터 $\theta$"**를 찾는 것을 목표로 합니다.